## Importamos el excel

Verificar Recursos del Sistema

In [1]:
import tensorflow as tf

print("GPUs disponibles:", len(tf.config.experimental.list_physical_devices('GPU')))


GPUs disponibles: 1


2024-06-29 17:59:52.289018: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-06-29 17:59:52.307118: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-06-29 17:59:52.307261: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [2]:
import torch

if torch.cuda.is_available():
    print("CUDA está disponible. Número de GPUs:", torch.cuda.device_count())
    print("Nombre de la GPU:", torch.cuda.get_device_name(0))
else:
    print("CUDA no está disponible.")


CUDA está disponible. Número de GPUs: 1
Nombre de la GPU: NVIDIA GeForce RTX 3050 Laptop GPU


In [3]:
import os
import multiprocessing
import torch
import tensorflow as tf

def check_cpus():
    print("Número de CPUs (os.cpu_count()):", os.cpu_count())
    print("Número de CPUs (multiprocessing.cpu_count()):", multiprocessing.cpu_count())

def check_gpu_pytorch():
    if torch.cuda.is_available():
        print("CUDA está disponible. Número de GPUs:", torch.cuda.device_count())
        print("Nombre de la GPU:", torch.cuda.get_device_name(0))
    else:
        print("CUDA no está disponible en PyTorch.")

def check_gpu_tensorflow():
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        print("GPUs disponibles en TensorFlow:", len(gpus))
        for gpu in gpus:
            print("Nombre de la GPU:", gpu.name)
    else:
        print("CUDA no está disponible en TensorFlow.")

if __name__ == "__main__":
    check_cpus()
    check_gpu_pytorch()
    check_gpu_tensorflow()


Número de CPUs (os.cpu_count()): 20
Número de CPUs (multiprocessing.cpu_count()): 20
CUDA está disponible. Número de GPUs: 1
Nombre de la GPU: NVIDIA GeForce RTX 3050 Laptop GPU
GPUs disponibles en TensorFlow: 1
Nombre de la GPU: /physical_device:GPU:0


Cargar y Preprocesar Datos

In [4]:
import pandas as pd

# Ruta al archivo Excel
file_path = '/home/jovyan/work/transcripciones_frases_final.xlsx'

# Cargamos el archivo Excel
df = pd.read_excel(file_path)

# Verificar que las columnas existen
assert 'Texto Original' in df.columns, "La columna 'Texto Original' no existe en el archivo Excel"
assert 'Texto Corregido' in df.columns, "La columna 'Texto Corregido' no existe en el archivo Excel"

df.head()


,Texto Original,Texto Corregido
0,esto es importante para cada adulto o cada per...,Esto es importante para cada adulto o cada per...
1,no salió todo como queríamos fue un buen año m...,"No salió todo como queríamos, fue un buen año...."
2,a lo largo de este top verás retratadas siete ...,A lo largo de este top verás retratadas siete ...
3,te prevengo querida audiencia los próximos min...,"Te prevengo, querida audiencia, los próximos m..."
4,número siete Benjamin Solari parravicini fue u...,Número siete: Benjamin Solari Parravicini fue ...


Dividir Datos en Entrenamiento y Validación

In [5]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(df, test_size=0.15, random_state=42)
train_df.head()

,Texto Original,Texto Corregido
785,"Bueno, yo comencé a grabar ese video, le da un...","Bueno, yo comencé a grabar ese video, le di un..."
1355,ahora las cosas han cambiado con las redes soc...,Ahora las cosas han cambiado con las redes soc...
528,Vale ahora hacemos Dios Dios qué ven mis ojos ...,"Vale, ahora hacemos... Dios. Dios. ¿Qué ven mi..."
1506,Pues bueno otra las funciones de tener un VPN ...,"Pues bueno, otra de las funciones de tener un ..."
582,"Por suerte nada más hicimos del número uno, si...","Por suerte nada más hicimos del número uno, si..."


Cargar el Modelo y el Tokenizador

Usaremos el modelos T5. Tokenizamos antes de introducir el texto en el modelo con T5Tokenizer 
Se puede usar 't5-base' o 't5-large' según lo preciso (más tiempo de entrenamiento) que queramos entrenar el modelo 

In [6]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Cargar el tokenizador y el modelo
model_name = 't5-small'  # Puedes usar 't5-small', 't5-base', 't5-large', etc.
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Preprocesar Datos

In [7]:
def preprocess_data(data, tokenizer, max_len=60):
    inputs = [f"corrija: {text}" for text in data['Texto Original']]
    targets = [text for text in data['Texto Corregido']]
    
    input_encodings = tokenizer(inputs, truncation=True, padding=True, max_length=max_len)
    target_encodings = tokenizer(targets, truncation=True, padding=True, max_length=max_len)
    
    for i in range(5):  # Verificar los primeros 5 pares
        print(f"Input {i}: {inputs[i]}")
        print(f"Target {i}: {targets[i]}")
        
    return {
        'input_ids': input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }

train_encodings = preprocess_data(train_df, tokenizer)
val_encodings = preprocess_data(val_df, tokenizer)


Input 0: corrija: Bueno, yo comencé a grabar ese video, le da un traguito a la copita, me sabía bien, pero no me caía bien.
Target 0: Bueno, yo comencé a grabar ese video, le di un traguito a la copita, me sabía bien, pero no me caía bien.
Input 1: corrija: ahora las cosas han cambiado con las redes sociales puesto que no hay una fuente de información única
Target 1: Ahora las cosas han cambiado con las redes sociales, puesto que no hay una fuente de información única.
Input 2: corrija: Vale ahora hacemos Dios Dios qué ven mis ojos qué ven mis ojos de momento no ven nada eso qué [ __ ] es vamos Azalea Azalea qué asco de bioma qué asco de bioma decepcionado Holden Holden no no per son este tipo de árboles eh
Target 2: Vale, ahora hacemos... Dios. Dios. ¿Qué ven mis ojos? ¿Qué ven mis ojos? De momento no ven nada. ¿Eso qué es? Vamos. Azalea. Azalea. Qué asco de bioma. Qué asco de bioma. Decepcionado. Holden. Holden. No, no, pero son este tipo de árboles, eh.
Input 3: corrija: Pues bueno 

Crear Dataset para PyTorch

In [8]:
import torch
from torch.utils.data import Dataset

class TextCorrectionDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx], dtype=torch.long) for key, val in self.encodings.items()}
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

train_dataset = TextCorrectionDataset(train_encodings)
val_dataset = TextCorrectionDataset(val_encodings)


Configurar y Entrenar el Modelo

In [9]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='/home/jovyan/results',  # Directorio de salida
    num_train_epochs=1,                 # Épocas
    per_device_train_batch_size=1,      # Tamaño del lote de entrenamiento
    per_device_eval_batch_size=1,       # Tamaño del lote de evaluación
    warmup_steps=500,                   # Pasos de calentamiento
    weight_decay=0.01,                  # Desintegración del peso
    logging_dir='/home/jovyan/logs',    # Directorio de logs
    evaluation_strategy="epoch",
    save_strategy="epoch",
    report_to="none",                   # Evitar el logging a WandB
    fp16=True                           # Entrenamiento en punto flotante de 16 bits
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.489900,0.397332


TrainOutput(global_step=1867, training_loss=0.9228888145409999, metrics={'train_runtime': 168.9885, 'train_samples_per_second': 11.048, 'train_steps_per_second': 11.048, 'total_flos': 29611305861120.0, 'train_loss': 0.9228888145409999, 'epoch': 1.0})

Generar Predicciones y Evaluar el Modelo

In [12]:
import torch
from datasets import load_metric
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def generate_predictions(model, tokenizer, dataset, max_length=60, device='cuda' if torch.cuda.is_available() else 'cpu'):
    model.to(device)
    model.eval()
    predictions = []
    for i in range(len(dataset)):
        inputs = tokenizer(dataset[i]['Texto Original'], return_tensors="pt", padding=True, truncation=True, max_length=max_length)
        inputs = {key: val.to(device) for key, val in inputs.items()}  # Mover los tensores a la GPU si está disponible
        outputs = model.generate(inputs['input_ids'], max_length=max_length, num_beams=4, early_stopping=True)
        pred_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        predictions.append(pred_text)
    return predictions

# Generar predicciones para el conjunto de validación
val_predictions = generate_predictions(model, tokenizer, val_df.to_dict(orient='records'))

# Métricas de evaluación
rouge = load_metric('rouge')
bleu = load_metric('bleu')

# Preparar referencias y predicciones para evaluación
references = val_df['Texto Corregido'].tolist()

# Calcular métricas
rouge_score = rouge.compute(predictions=val_predictions, references=references)
bleu_score = bleu.compute(predictions=[pred.split() for pred in val_predictions],
                          references=[[ref.split()] for ref in references])

print("ROUGE score:", rouge_score)
print("BLEU score:", bleu_score)

# Para métricas de clasificación
predicted_labels = [pred.split()[0] for pred in val_predictions]  # Ajusta según tu caso
true_labels = val_df['Texto Corregido'].tolist()

accuracy = accuracy_score(true_labels, predicted_labels)
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predicted_labels, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


Crear y Visualizar la Matriz de Confusión

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Convertir a etiquetas binarias (correcto/incorrecto)
true_labels_binary = [1 if true == pred else 0 for true, pred in zip(true_labels, val_predictions)]
predicted_labels_binary = [1 if true == pred else 0 for true, pred in zip(true_labels, val_predictions)]

# Crear la matriz de confusión
cm = confusion_matrix(true_labels_binary, predicted_labels_binary, labels=[1, 0])
cmd = ConfusionMatrixDisplay(cm, display_labels=["Correcto", "Incorrecto"])

# Graficar la matriz de confusión
cmd.plot()
plt.title("Matriz de Confusión - Corrección de Texto")
plt.xlabel("Etiqueta Predicha")
plt.ylabel("Etiqueta Verdadera")
plt.show()
